## RAG flow

#### Setup for Qdrant

First, install the Qdrant client with FastEmbed support:

```bash
pip install -q "qdrant-client[fastembed]>=1.14.2"
```
Then, run Qdrant in Docker:

```bash
docker pull qdrant/qdrant

docker run -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant

In [13]:
import json
import pandas as pd
import time
from tqdm.auto import tqdm
from qdrant_client import QdrantClient, models
from google import genai
from groq import Groq
from dotenv import load_dotenv
import os

# Qdrant client
client = QdrantClient("http://localhost:6333")

# Load the environment variables from the .env file
load_dotenv()

# Retrieve Groq API key
# https://console.groq.com/keys
# 1000 credits for gpt-oss
groq_api_key = os.getenv('GROQ_API_KEY')

# Retrieve Google API key
# free api key https://aistudio.google.com/app/apikey
# 250 free RPD for 2.5-flash, 1000 for 2.5-flash-lite
google_api_key = os.getenv('GOOGLE_API_KEY')

## Load data - 200 sample questions due to api credit limits 

In [14]:
plants_data = pd.read_csv("../data/plants_data.csv")
documents = plants_data.to_dict(orient='records')

df_question = pd.read_csv("../data/ground-truth-retrieval-5q.csv")
df_sample = df_question.sample(n=200, random_state=2)
ground_truth_sample = df_sample.to_dict(orient='records')

In [15]:
df_sample.head()

,id,question
817,163,What kind of environmental conditions does Mon...
658,131,"Where is Goeppertia picturata originally from,..."
969,193,What characteristics make Platycerium bifurcat...
553,110,What alternate name is sometimes used for Drac...
158,31,In what year was Alocasia scalprum officially ...


## Hybrid search functions

In [16]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [17]:
collection_name = "rag-project-sparse-and-dense"
# Create the collection with both vector types
if client.collection_exists(collection_name):
    client.delete_collection(collection_name)    
client.create_collection(
    collection_name=collection_name,
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        "jina-small": models.VectorParams(
            size=512,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [18]:
points = []

for i, doc in enumerate(documents):
    text = doc['name'] + ' ' + doc['summary'] + ' ' + doc['cultivation']+ ' ' + doc['toxicity']
    vector = {
                "jina-small": models.Document(
                    text=text,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=text, 
                    model="Qdrant/bm25",
                ),
    }
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [19]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

...

portuguese.txt: 0.00B [00:00, ?B/s]

spanish.txt: 0.00B [00:00, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
def multi_stage_search(query, limit = 5):
    """
    Perform a hybrid multi-stage search combining BM25 keyword search 
    with semantic prefetch using Jina embeddings. 
    Prefetch retrieves 10× the requested results for improved reranking, 
    then returns the top matches with payloads.

    Args:
        query (str): Search query text.
        limit (int, optional): Number of final results to return. Defaults to 5.

    Returns:
        list[models.ScoredPoint]: Ranked search results with payload data.
    """
    results = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25", 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results

In [21]:
prompt_template1 = """
You are a knowledgeable and friendly plant specialist.
Your expertise covers house plants, their care and toxicity.
Answer the QUESTION based on the CONTEXT from our plants database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template1 = """
plant name: {name}
summary: {summary}
cultivation: {cultivation}
toxicity: {toxicity}
""".strip()


In [22]:
prompt_template2 = """
You are a knowledgeable and friendly plant specialist.
Your expertise covers houseplants, their care, and toxicity.

Answer the QUESTION using only the information provided in CONTEXT. 
If the CONTEXT does not contain the answer, say so clearly. 
Do not add outside knowledge or assumptions. 
Keep your answer clear, accurate, and concise.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template2 = """
Plant: {name}
Summary: {summary}
Cultivation: {cultivation}
Toxicity: {toxicity}
""".strip()

def build_prompt(query, search_results, prompt_template, entry_template):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

## Groq API

In [17]:
def gpt_oss_answer(content, model="openai/gpt-oss-20b"):
  """
    Generate GPT-OSS model response from Groq Api.

    Args:
        content (str): Prompt for the model.
        model (str, optional): Model name. Defaults to "openai/gpt-oss-20b".

    Returns:
        str: Concatenated response text.
  """
  client = Groq(api_key=groq_api_key)
  completion = client.chat.completions.create(
      model=model,
      messages=[
        {
          "role": "system",
          "content": content
        }
      ],
      temperature=1,
      max_completion_tokens=8192,
      top_p=1,
      reasoning_effort="medium",
      stream=True,
      stop=None
  )
  joined_answer = ''
  
  for chunk in completion:

      chunk_answer = chunk.choices[0].delta.content or ""
      joined_answer = joined_answer + chunk_answer
    
  return joined_answer

In [20]:
def rag_groq(query, prompt_template, entry_template):
    """
    Run RAG with Groq GPT-OSS model.

    Args:
        query (str): User query to answer.
        prompt_template (str): Template for constructing the full prompt.
        entry_template (str): Template for formatting retrieved entries.

    Returns:
        str: Model-generated answer.
    """
    search_results = multi_stage_search(query)
    search_results_list = []

    for i in search_results.points:

        search_results_list.append(i.payload)

    prompt = build_prompt(query, search_results_list, prompt_template, entry_template)
    answer = gpt_oss_answer(prompt)
    
    return answer

In [21]:
question = "What plants are toxic?"
answer = rag_groq(question, prompt_template1, entry_template1)
print(answer)

**Plants that are confirmed toxic in the context above**

| Plant | Toxic parts & substances | Pets/people at risk |
|------|------------------------|--------------------|
| **Philodendron hederaceum** | Calcium oxalate crystals in all parts; toxic when ingested in large amounts. | Causes oral irritation, swelling, vomiting, and can deposit crystals in kidneys. |
| **Anthurium clarinervium** | Insoluble calcium oxalate crystals in every part (roots, stems, leaves, flowers, seeds). | Can cause oral irritation, pain, swelling, excessive drooling, vomiting and difficulty swallowing in cats, dogs, and horses. |
| **Jatropha podagrica** | All parts, especially seeds, contain purgative oil and toxalbumin (curcin), similar to ricin. | Highly toxic – ingestion can be fatal. |
| **Kalanchoe daigremontiana** | All parts contain the steroid toxin *daigremontianin*. | Toxic to animals and humans. |

**Plants with uncertain or unconfirmed toxicity**

| Plant | Status |
|------|-------|
| **Crassula

In [23]:
query = "What plants are toxic?"
search_results = multi_stage_search(query)
search_results_list = []
for i in search_results.points:
    search_results_list.append(i.payload)
prompt = build_prompt(query, search_results_list, prompt_template2, entry_template2)

print(prompt)

You are a knowledgeable and friendly plant specialist.
Your expertise covers houseplants, their care, and toxicity.

Answer the QUESTION using only the information provided in CONTEXT. 
If the CONTEXT does not contain the answer, say so clearly. 
Do not add outside knowledge or assumptions. 
Keep your answer clear, accurate, and concise.

QUESTION: What plants are toxic?

CONTEXT:
Plant: Philodendron hederaceum
Summary: Philodendron hederaceum,  the heartleaf philodendron (syn. Philodendron scandens) is a species of flowering plant in the family Araceae, native to Central America and the Caribbean which is common in the houseplant trade. Philodendron hederaceum var. hederaceum, the "velvet philodendron," is a subspecies which is in the houseplant trade under its previous name of Philodendron micans. While toxic under certain conditions, it is also under current review for numerous health benefits.
Cultivation: No data available
Toxicity: Parts of the plant are known to contain calcium 

## LLM as a judge prompt

In [24]:
llm_judge_prompt = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [25]:
prompt = llm_judge_prompt.format(question=question, answer_llm=answer)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: What plants are toxic?
Generated Answer: **Plants that are confirmed toxic in the context above**

| Plant | Toxic parts & substances | Pets/people at risk |
|------|------------------------|--------------------|
| **Philodendron hederaceum** | Calcium oxalate crystals in all parts; toxic when ingested in large amounts. | Causes oral irritation, swelling, vomiting, and can deposit crystals in kidneys. |
| **Anthurium clarinervium** | Insoluble calcium oxalate crystals in every part (roots, stems, leaves, flowers, seeds). | Can cause oral irritation, pain, swelling, excessive drooling, vomiting and difficulty swallowing in cats, dogs, and horses. |
| **Jatropha podagrica** | All parts, especial

## GPT-OSS evaluation

In [27]:
evaluations = []

In [ ]:
for record in tqdm(ground_truth_sample):
    question = record['question']
    answer_llm = rag_groq(question, prompt_template2, entry_template2) 

    prompt = llm_judge_prompt.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = gpt_oss_answer(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/75 [00:00<?, ?it/s]

In [27]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])


## Save gpt-oss results to csv

In [ ]:
df_eval.to_csv('../data/rag_eval_gpt_oss_prompt1.csv', index=False)
#df_eval.to_csv('../data/rag_eval_gpt_oss_prompt2.csv', index=False)

In [35]:
df_eval.to_csv('../data/rag_eval_gpt_oss_prompt2_part3.csv', index=False)

## Google api

In [ ]:
def google_api(content, model="gemini-2.5-flash-lite"):
    """
    Generate a response from a Google Gemini model.

    Args:
        content (str): Prompt or instruction for the model.
        model (str, optional): Model name. Defaults to "gemini-2.5-flash-lite".

    Returns:
        str: Generated response text.
    """
    client = genai.Client(api_key=google_api_key)

    response = client.models.generate_content(
        model=model, contents=content
    )
    return response.text

In [30]:
def rag_google(query, prompt_template, entry_template):
    """
    Run RAG with Google Gemini model.

    Args:
        query (str): User query to answer.

    Returns:
        str: Model-generated answer.
    """
    search_results = multi_stage_search(query)
    search_results_list = []

    for i in search_results.points:

        search_results_list.append(i.payload)

    prompt = build_prompt(query, search_results_list, prompt_template, entry_template)
    answer = google_api(prompt)
    
    return answer

In [33]:
print(rag_google("Is monstera toxic", prompt_template1, entry_template1))

Yes, Monstera deliciosa is moderately toxic to both cats and dogs. This is due to the presence of insoluble calcium oxalate crystals, which can cause injury to the mouth, tongue, and digestive tract. Direct contact with the plant can also cause dermatitis on a pet's skin.


In [34]:
print(rag_google("which plants are toxic", prompt_template1, entry_template1))

The following plants are toxic:

*   **Philodendron hederaceum:** Parts of the plant contain calcium oxalate crystals. Ingesting large quantities can lead to kidney issues and cardiac-related problems in humans. It is also toxic to mice and rats. For animals, it can cause oral irritation, a swollen mouth, lips, and tongue, drooling, vomiting, and difficulty swallowing.
*   **Anthurium clarinervium:** All plants in the Anthurium genus are toxic to cats, dogs, and horses. All parts of the plant contain insoluble calcium oxalate crystals, which can cause oral irritation, pain, swelling, excessive drooling, vomiting, and difficulty swallowing.
*   **Jatropha podagrica:** All parts of this plant are considered toxic, especially the seeds. The main toxins are a purgative oil and a phytotoxin (curcin).


## Gemini evaluation

In [62]:
evaluations = []

In [ ]:
for record in tqdm(ground_truth_sample):

    question = record['question']
    answer_llm = rag_google(question) 

    prompt = llm_judge_prompt.format(
        question=question,
        answer_llm=answer_llm
    )
    #evaluate using groq!
    evaluation = gpt_oss_answer(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))
    #for TPM (tokens per minute) limits
    time.sleep(5)

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

## Summary

In [1]:
gpt_oss_prompt1 = pd.read_csv('../data/rag_eval_gpt_oss_prompt1.csv')
gpt_oss_prompt2 = pd.read_csv('../data/rag_eval_gpt_oss_prompt2.csv')
gemini_flash_2_5_lite_prompt1 = pd.read_csv('../data/rag_eval_gemini_flash_2_5_lite_prompt1.csv')
gemini_flash_2_5_lite_prompt2 = pd.read_csv('../data/rag_eval_gemini_flash_2_5_lite_prompt2.csv')

In [9]:
print("GPT OSS PROMPT1", gpt_oss_prompt1.shape)
print(gpt_oss_prompt1.relevance.value_counts(normalize=True))
print("--------------------------------------------------")
print("GPT OSS PROMPT2", gpt_oss_prompt2.shape)
print(gpt_oss_prompt2.relevance.value_counts(normalize=True))
print("--------------------------------------------------")
print("GEMINI FLASH 2.5 LITE PROMPT1", gemini_flash_2_5_lite_prompt1.shape)
print(gemini_flash_2_5_lite_prompt1.relevance.value_counts(normalize=True))
print("--------------------------------------------------")
print("GEMINI FLASH 2.5 LITE PROMPT2", gemini_flash_2_5_lite_prompt2.shape)
print(gemini_flash_2_5_lite_prompt2.relevance.value_counts(normalize=True))

GPT OSS PROMPT1 (200, 7)
relevance
RELEVANT           0.905
PARTLY_RELEVANT    0.085
NON_RELEVANT       0.010
Name: proportion, dtype: float64
--------------------------------------------------
GPT OSS PROMPT2 (200, 7)
relevance
RELEVANT           0.83
PARTLY_RELEVANT    0.13
NON_RELEVANT       0.04
Name: proportion, dtype: float64
--------------------------------------------------
GEMINI FLASH 2.5 LITE PROMPT1 (200, 7)
relevance
RELEVANT           0.705
PARTLY_RELEVANT    0.255
NON_RELEVANT       0.040
Name: proportion, dtype: float64
--------------------------------------------------
GEMINI FLASH 2.5 LITE PROMPT2 (200, 7)
relevance
RELEVANT           0.67
PARTLY_RELEVANT    0.23
NON_RELEVANT       0.10
Name: proportion, dtype: float64


In [7]:
print(gpt_oss_prompt1[gpt_oss_prompt1.relevance == 'NON_RELEVANT']['question'].to_list())

['In which regions is Jatropha podagrica commonly cultivated as an indoor plant, despite its native habitat being Central America and southern Mexico?', 'What are the common names associated with Dracaena pethera?']


In [10]:
gpt_oss_prompt1[gpt_oss_prompt1.relevance == 'NON_RELEVANT']['answer'].to_list()

['**Answer:**  \nJatropha\u202fpodagrica is grown as an indoor plant in **many parts of the world**.',
 'Dracaena\u202fpethera is commonly known as the **star sansevieria** or the **snake plant**.']

In [11]:
gpt_oss_prompt1[gpt_oss_prompt1.relevance == 'NON_RELEVANT']['explanation'].to_list()

['The generated answer does not specify any particular regions where Jatropha podagrica is cultivated indoors; it merely states that it is grown in many parts of the world, which fails to address the question about specific regions.',
 'The answer lists common names that actually belong to other Dracaena species (e.g., snake plant, star sansevieria). Dracaena pethera does not use these names, so the answer is not relevant to the question.']

In [23]:
query='In which regions is Jatropha podagrica commonly cultivated as an indoor plant, despite its native habitat being Central America and southern Mexico?'
search_results = multi_stage_search(query)
search_results_list = []
for i in search_results.points:
        search_results_list.append(i.payload)
prompt = build_prompt(query, search_results_list, prompt_template2, entry_template2)
print(prompt)

You are a knowledgeable and friendly plant specialist.
Your expertise covers houseplants, their care, and toxicity.

Answer the QUESTION using only the information provided in CONTEXT. 
If the CONTEXT does not contain the answer, say so clearly. 
Do not add outside knowledge or assumptions. 
Keep your answer clear, accurate, and concise.

QUESTION: In which regions is Jatropha podagrica commonly cultivated as an indoor plant, despite its native habitat being Central America and southern Mexico?

CONTEXT:
Plant: Jatropha podagrica
Summary: Jatropha podagrica is a species of flowering, caudiciform succulent plant in the spurge family, Euphorbiaceae, aligning it closely with related genera such as Croton, Euphorbia and Ricinus (castor bean), among others. It is native to the neotropics of Central America and southern Mexico, but is grown as an ornamental plant in many parts of the world due to its unusual appearance and mature caudex development. Common names for the species include gout-

In [24]:
query='What are the common names associated with Dracaena pethera?'
search_results = multi_stage_search(query)
search_results_list = []
for i in search_results.points:
        search_results_list.append(i.payload)
prompt = build_prompt(query, search_results_list, prompt_template2, entry_template2)
print(prompt)

You are a knowledgeable and friendly plant specialist.
Your expertise covers houseplants, their care, and toxicity.

Answer the QUESTION using only the information provided in CONTEXT. 
If the CONTEXT does not contain the answer, say so clearly. 
Do not add outside knowledge or assumptions. 
Keep your answer clear, accurate, and concise.

QUESTION: What are the common names associated with Dracaena pethera?

CONTEXT:
Plant: Dracaena pethera
Summary: Dracaena pethera, synonym Sansevieria kirkii, also known as the star sansevieria or the snake plant, is a succulent plant native to Tanzania and the surrounding region in East Africa.
Cultivation: No data available
Toxicity: No data available

Plant: Dracaena reflexa
Summary: Dracaena reflexa (commonly called song of India or song of Jamaica) is a tree native to Mozambique, Madagascar, Mauritius, and other nearby islands of the Indian Ocean. It is widely grown as an ornamental plant and houseplant, valued for its richly coloured, evergreen 

In [ ]:
##comment
#actually non relevant answers are correct based on the context.
# I notce that sometimes, the model answersthe question based on its own knowledge (if there is lack of information in the context)
# but it is not a problem in my case


In [ ]:
print(gpt_oss_prompt2[gpt_oss_prompt2.relevance == 'NON_RELEVANT']['question'].to_list())

['What type of environment does the scalloped laceleaf thrive in?', 'What characteristics make the foliage of Begonia venosa striking?', 'How is the Moon Cactus typically propagated or cultivated for home gardening?', 'What unique characteristics define the family Didymochlaenaceae?', 'Is Philodendron gigas known for producing flowers?', 'What should I know about the growth habit of Campanula isophylla?', 'Can you explain why Monstera adansonii is also referred to as the Swiss cheese plant?', 'What type of environment is recommended for growing this species of plant?']


In [26]:
print(gpt_oss_prompt2[gpt_oss_prompt2.relevance == 'NON_RELEVANT']['answer'].to_list())

['The context does not provide specific information about the preferred environment for Anthurium\u202fcrenatum (the scalloped laceleaf). Therefore, I cannot say what type of environment it thrives in.', 'The provided context does not include any specific details about what makes the foliage of Begonia\u202fvenosa striking.', 'I’m sorry, but the provided context does not contain information on how Moon Cactus (Gymnocalycium mihanovichii) is propagated or cultivated for home gardening.', 'The context does not provide any information about the distinctive characteristics that define the family Didymochlaenaceae.', 'Yes. Philodendron gigas is described in the context as a species of *flowering plant* in the family Araceae, meaning it produces flowers.', 'I’m sorry, but the provided context does not contain any information about the growth habit of **Campanula isophylla**.', 'Monstera\u202fadansonii is called the **Swiss cheese plant** simply because that common name is used for it (and al

In [28]:
query='What unique characteristics define the family Didymochlaenaceae?'
search_results = multi_stage_search(query)
search_results_list = []
for i in search_results.points:
        search_results_list.append(i.payload)
prompt = build_prompt(query, search_results_list, prompt_template2, entry_template2)
print(prompt)


You are a knowledgeable and friendly plant specialist.
Your expertise covers houseplants, their care, and toxicity.

Answer the QUESTION using only the information provided in CONTEXT. 
If the CONTEXT does not contain the answer, say so clearly. 
Do not add outside knowledge or assumptions. 
Keep your answer clear, accurate, and concise.

QUESTION: What unique characteristics define the family Didymochlaenaceae?

CONTEXT:
Plant: Didymochlaena
Summary: Didymochlaena is a genus of ferns. In the Pteridophyte Phylogeny Group classification of 2016 (PPG I), it is the only genus in the family Didymochlaenaceae. Alternatively, the family may be placed in a very broadly defined family Polypodiaceae sensu lato as the subfamily Didymochlaenoideae.


Cultivation: No data available
Toxicity: No data available

Plant: Kalanchoe daigremontiana
Summary: Kalanchoe daigremontiana, formerly known as Bryophyllum daigremontianum and commonly called mother of thousands, alligator plant or Mexican hat plant